In [5]:
import warnings, csv, os, sys, json
import pandas as pd
import json_repair
from typing import List
warnings.filterwarnings("ignore")
from langchain_openai import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate
)
from langchain_community.utils.openai_functions import (
    convert_pydantic_to_openai_function,
)
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

In [2]:
def write_csv(output_file, data):
    with open(output_file, 'a', newline='', encoding='utf-8') as outfile:  
        writer = csv.writer(outfile)
        writer.writerow(data)

In [3]:
def write_txt(txt_path, data):
    with open(txt_path, 'a') as txf:
        txf.write(data)

In [6]:
#variables

model_name = 'gpt-4-1106-preview'  #gpt model name
audience = ' US owners of 1 dog'  #audience
output_file = 'run2.csv'
txt_file = 'output.txt'

#you can change questions and traits/count as you want, this part has become dynamic in this notebook.
questions = [
"How old is your dog (in # of years)?",
"What is your dog's primary breed?",
"Approximately how much does your dog weigh (in pounds)?",
"What is your dog's name?",
"What was the inspiration for that name?",
"What is the most common nickname (if any) that you use for your dog?",
"When did you get your dog? (Before 2020, In 2020, After 2020)",
"Did you adopt your dog? (yes/no)",
"How old was your dog when you got him/her? Provide estimated number of months old.",
"How did you feel the first time you met your dog?",
"How has your dog changed your outlook on life?",
"On a scale of 1-5, where 1 is not at all important and 5 is very important, how important is your dog's happiness to your overall well-being?",
"Choose the term that best describes your dog's relationship with you.(Best Friend,Loyal companion,Family Member,Protector,None of these apply)",
"Why did you choose that term?",
"What is the most recent command or trick your dog has learned, and how long did it take to train?",
"How many hours a day does your dog typically sleep and where does your dog usually sleep in your home?",
"Have you shopped for your dog online in the past year? (yes/no)",
"Over the past year, approximately how much did you spend on your dog? Provide total $ amount.",
"Over the past year, approximately what percent of the money you spent on your dog was paid to your veterinarian? Choose a percent from 0% to 100%.",
"Over the past year, approximately what percent of the money you spent on your dog was on dog food? Choose a percent from 0% to 100%.",
"In March, approximately much money did you spend on your dog? Provide estimated total $ amount.",
"How did the amount of money you spent on your dog in March compare with February?(Spent about the same in March,I'm not sure,Spent more in March,Spent less in March)",
"How often do you purchase dog food? (Bi-weekly,Monthly,Less frequently than monthly,Weekly)",
"Do you primarily feed your dog dry food, wet food, or a mix?",
"Which of the following statements best describes your level of familiarity of your dog's primary food brand?(I know a lot about the brand,I know just the basics about the brand,I'm not very familiar with the brand)",
"On a scale of 1-5, where 1 is not at all important and 5 is very important, how important is the following in choosing your primary dog food brand? Quality of ingredients",
"Price",
"Nutritional value of the dog food",
"Brand reputation",
"How your dog likes the taste",
"Your veterinarian's recommendation",
"Special dietary needs",
"What dog food brand do you give your dog the most?",
"What are the main 2-3 reasons you choose this brand?",
"On a scale of 1-5, where 1 is not at all satisfied and 5 is very satisfied, how satisfied are you with the following for your primary dog food brand? Quality of ingredients",
"Price",
"Nutritional value of the dog food",
"Brand reputation",
"How your dog likes the taste",
"Your veterinarian's recommendation",
"Special dietary needs",
"How many times a day does your dog typically go out for a walk?(0,1,2,3,4,5,more than 5,not sure)",
"How (if at all) have recent events affected how you interact with your dog?(No change,Not sure,I interact more,I interact less)",
"What events are affecting your interaction level most?",
"Are you planning to watch the Summer Olympics with your dog?(No,Yes,Not Sure)",
"Are you planning to travel this Spring with your dog?(No,Yes,Not Sure)",
"If planning to travel this Spring with your dog, which is your preferred method of transportation? Select the best option.(Bus,Car,N/A-I'm not planning to travel with my dog this Spring,Other)",
"What is your gender?",
"What is your age?",
"How many people, including yourself, live in your household?(1,2,3,5 or more)",
"What is the highest level of education you have completed?(Associate degree, Bachelor's degree,Graduate degree, High school diploma or equivalent, Some college, no degree)",
"What is your current employment status?(Employed full-time,Employed part-time,Retired,Self-employed,Unable to work,Unemployed)",
"What is your total annual household income before taxes?($100,000 to $149,000,$150,000 or more,$25,000 to $49,999,$50,000 to $74,999, Less than $25,000, Prefer not to say)",
"In what city do you currently reside?",
"In what state do you currently reside?"
]

traits_and_counts = [
    ('Men aged 60+', 17)
]


In [5]:
# questions = [
#     'How happy are you with your life on a scale of 1-5 where 1 is very unhappy and 5 is very happy? (#)',
#     'What are the few tech tools that you use daily?',
#     'What is your profession?',
# ]

# traits_and_counts = [
#     ('Men being currently employed', 4),
#     ('Women being currently employed', 3)
# ]


In [10]:
# This block of code is not needed, it is there to know how function calling works.

# To learn more about function calling, check this out https://www.datacamp.com/tutorial/open-ai-function-calling-tutorial
#Answer is the class which contains response given by each user, it also contain traits to know which person answered the questions.
#llm is inteligent enough to give unique response each time based on the given system and user prompt, we should not worry about it.
class Answer(BaseModel):
    """Elements of JSON structure that is extracted from User Prompts. """

    type: str = Field(description="The type command extracted from User Prompt.")
    andSelect: list = Field(description="The list of conditions based on select and where statement extracted from User Prompt.")
    transformColumn: str = Field(description="The column that needs to be transformed extracted from User Prompt.")
    transformOperation: str = Field(description="The transformation operation needs to be done on column given in User Prompt.")
    transformValue: str = Field(description="The value that needs to be transformed.")
    mode: str = Field(description="Mode of the user prompt")
    selectExpr: str = Field(description="Count of values in select statement that needs to be transformed")
    headerName: str = Field(description="Column name given in User Prompt")
    
    
    
#list of answers, the formatted list that the llm model will generate which contains dictionaries of Answer class
#llm model will make sure that each dictionary in the list is unique from each other as it will treat the list like list of human responses.
# class Answers(BaseModel):
#     """Human-like survey answers to tell user."""

#     answer: List[Answer]

In [11]:
openai_functions = [convert_pydantic_to_openai_function(Answer)]

In [12]:
openai_functions

[{'name': 'Answer',
  'description': 'Elements of JSON structure that is extracted from User Prompts. ',
  'parameters': {'type': 'object',
   'properties': {'type': {'description': 'The type command extracted from User Prompt.',
     'type': 'string'},
    'andSelect': {'description': 'The list of conditions based on select and where statement extracted from User Prompt.',
     'type': 'array',
     'items': {}},
    'transformColumn': {'description': 'The column that needs to be transformed extracted from User Prompt.',
     'type': 'string'},
    'transformOperation': {'description': 'The transformation operation needs to be done on column given in User Prompt.',
     'type': 'string'},
    'transformValue': {'description': 'The value that needs to be transformed.',
     'type': 'string'},
    'mode': {'description': 'Mode of the user prompt', 'type': 'string'},
    'selectExpr': {'description': 'Count of values in select statement that needs to be transformed',
     'type': 'string

In [53]:
system_message = """
You are a JSON extracting bot from the given User prompt. You need to return JSON by extracting useful elements from the User Prompt.
User prompt generally is statements of SQL commands.
Here are few examples of User prompt and the JSON format that needs to be extracted from the User prompt:

User Prompt: Select rows where column 9 contains USD. Then, prepend 'ID-' to the beginning of column 2.
Answer:
	"type": "SelectAndTransformCommand",
	"andSelect": [
		"9 containsIgnoreCase USD"
	],
	"transformColumn": "2",
	"transformOperation": "prepend",
	"transformValue": "ID-",
	"mode": "Modify"

User Prompt: change column 9 to this or last working date.
Answer:
	"type": "SelectAndTransformCommand",
	"transformColumn": "9",
	"transformOperation": "thisOrLastWorkingDate",
	"mode": "Modify"

User Prompt: Remove all instances of "0.000000" from within Column 1
Answer:
	"type": "SelectAndTransformCommand",
 	"selectExpr": "ALL",
	"transformColumn": "1",
	"transformOperation": "removeAllX",
	"transformValue": "0.000000",
	"mode": "Modify"

User Prompt: Where Column 78 is not blank, keep the first three characters
Answer:
	"type": "SelectAndTransformCommand",
	"andSelect": [
		"78 notEquals "
	],
	"transformColumn": "78",
	"transformOperation": "keepFirstXChars",
	"transformValue": "3",
	"mode": "Modify"

User Prompt: Round all numerical information within column 9 to two decimal places
Answer:
	"type": "SelectAndTransformCommand",
	"selectExpr": "ALL",
	"transformColumn": "9",
	"transformOperation": "rounding",
	"transformValue": "2",
	"mode": "Modify"

User Prompt: Add a new column "One" with value 1 for all rows
Answer:
	"type": "AddColumnCommand",
	"headerName": "One",
 	"selectExpr": "ALL",
	"value": "1"

"""

In [54]:
#bind openai_functions to llm for function calling
llm = ChatOpenAI(
    temperature=0,
    model_name=model_name
).bind(functions=openai_functions)
# llm = ChatOpenAI(
#     temperature=0,
#     model_name=model_name
# )

In [55]:
#chat template
prompt = ChatPromptTemplate.from_messages(
    [("system", system_message), ("user", "{input}")]
)

In [56]:
#chain using prompt, llm and parser
#it uses the prompt with llm and generate the answer based on the parser

# chain = prompt | llm | parser

chain = prompt | llm 

In [58]:
def output_parser(response):
    try:
        return eval(response.additional_kwargs['function_call']['arguments'])
    except Exception as e:
        try:
            return json_repair.loads(response.additional_kwargs['function_call']['arguments'])
        except Exception as e:
            return None

In [61]:
user_prompt = 'Where Column 77 is not equal to " ", then append "Sell" to Column 77'

input_message = f"Generate JSON format from given User Prompt.\n"
input_message += f'This is User Prompt: {user_prompt}\n'
res = chain.invoke({"input": input_message})

In [62]:
output_parser(res)

{'type': 'SelectAndTransformCommand',
 'andSelect': ['77 notEquals " "'],
 'transformColumn': '77',
 'transformOperation': 'append',
 'transformValue': 'Sell',
 'mode': 'Modify'}